In [1]:
#jaccard distance --> bad because ignore ratings
#cosine distance --> treat missing values as negative
#pearson distance/centred cosin distance --> good

In [1]:
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import correlation

In [2]:
data=pd.read_csv("/home/angps/Documents/DSC3216 (1)/project/the-movies-dataset/ratings_small.csv")
data.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [3]:
u = data.groupby('userId')['rating'].count()
user = u.sort_values(ascending = False)

m = data.groupby('movieId')['rating'].count()
movie = m.sort_values(ascending = False)

In [4]:
joined = data.join(user, on='userId', how = 'inner', rsuffix='_r')
joined = joined.join(movie, on='movieId', how = 'inner', rsuffix = '_r')

ratings=pd.crosstab(joined.userId, joined.movieId, joined.rating, aggfunc=np.sum)

In [5]:
ratings = ratings.fillna(value=0)
ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from sklearn import cross_validation as cv
train, test = cv.train_test_split(ratings, test_size=0.20)

/home/angps/anaconda3/envs/angps/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
test.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
284,3.0,0.0,3.0,0.0,4.0,5.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
503,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
train.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
295,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
445,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
def distance_train(user1,user2):
        try:
            user1Ratings = train.transpose()[user1]
            user2Ratings = train.transpose()[user2]
            distance = correlation(user1Ratings,user2Ratings)
        except: 
            distance = np.NaN
        return distance

In [10]:
#for test
def distance(testuser,trainuser):
        try:
            user1Ratings = test.transpose()[testuser]
            user2Ratings = train.transpose()[trainuser]
            distance = correlation(user1Ratings,user2Ratings)
        except: 
            distance = np.NaN
        return distance

In [11]:
def get_k_nearest(new_userid,k):
    table = pd.DataFrame(train.index)
    table["dist"] = table["userId"].apply(lambda x: distance(new_userid,x))   #calc distance of each user to the new user
    K_nearest_users = table.sort_values(["dist"],ascending=True)[:k]  #get lowest k distance (highest similarity)
    K_nearest_users.set_index("userId",inplace=True)
    return K_nearest_users

def ratings_of_k_nearest(new_userid,k):
    k_nearest = get_k_nearest(new_userid,k)
    rating = train[train.index.isin(k_nearest.index)]  # index is the user id. so get the movie ratings of all users in the k nearest users
    return rating

def get_ratings(train_user_id,movieid):
    return train.at[train_user_id,movieid]
def pred_ratings(new_userid,k):  #using weighted average
    '''Input new user ID and k, the number of neighours selected'''
    if k<20:
        return 'Please choose a higher k'
    else:
        k_nearest = get_k_nearest(new_userid,k)   #get k nearest users
        rating = ratings_of_k_nearest(new_userid,k)   #get ratings of the k nearest users
        merged = k_nearest.merge(rating,left_index=True,right_index=True) #combine to find the ratings of each movie for each k nearest users    
        pred = pd.DataFrame(rating.columns)
        pred.set_index("movieId",inplace=True)
        pred["ratings"] = 0.0
    for i in range(1,len(merged.columns)):
        col = merged.columns[i]
        valid_col = [t for t in merged.index if merged.loc[t, col]!=0]
        if len(valid_col)<5:   #can adjust how many valid ratings
            pred.at[col,'ratings'] = 0
        else:
            temp = merged.loc[valid_col,['dist',col]]
            temp['product'] = temp.prod(1)
            pred.at[col,'ratings'] = temp['product'].sum()/temp['dist'].sum()
    pred = pred.sort_values(["ratings"],ascending=False)
    return pred

def recommend_top_n(new_userid,k,top_n):
    if top_n == 0:
        return pred_ratings(new_userid,k).set_index("movieId")
    else:
        return pred_ratings(new_userid,k)[:top_n].set_index("movieId")
            

In [12]:
pred_ratings(488,20)

,ratings
movieId,
109374,4.800704
1214,4.695750
1201,4.662123
296,4.661252
47,4.586038
2692,4.572708
4878,4.545040
4011,4.533757
858,4.528001


In [17]:
real = pd.DataFrame(test.loc[488])
#print (real.head(3))
predicted = pd.DataFrame(pred_ratings(488, 20))
#print (predicted.head(3))
combine = real.merge(predicted,left_index=True,right_index=True)
combine = combine[(combine[488]!=0) & (combine['ratings']!=0)]
combine.columns = ['Real','Predicted']
combine['Errors'] = abs(combine['Real']-combine['Predicted'])

real_mean = combine['Real'].mean()
predicted_mean = combine['Predicted'].mean()
def accuracy(df):
    if df['Predicted'] > predicted_mean:
        if df['Real'] > real_mean:
            return 'True Positive'
        else:
            return 'False Positive'
    else:
        if df['Real'] < real_mean:
            return 'True Negative'
        else:
            return 'False Negative'
    
confusion_matrix = pd.DataFrame(0,['Like', 'Dislike'],['Predicted Like','Predicted Dislike'])

In [18]:
def cm(df):
    for user in test.index:
        real = pd.DataFrame(test.loc[user])
        predicted = pd.DataFrame(pred_ratings(user,20))
        combine = real.merge(predicted,left_index=True,right_index=True)
        combine = combine[(combine[user]!=0) & (combine['ratings']!=0)]
        combine.columns = ['Real','Predicted']
        real_mean = combine['Real'].mean()
        predicted_mean = combine['Predicted'].mean()
        combine['Accuracy'] = combine.apply(accuracy,1)
        if 'True Positive' in combine['Accuracy'].value_counts():
            df.loc['Like','Predicted Like'] += combine['Accuracy'].value_counts()['True Positive']
        if 'False Negative' in combine['Accuracy'].value_counts():
            df.loc['Like','Predicted Dislike'] += combine['Accuracy'].value_counts()['False Negative']
        if 'False Positive' in combine['Accuracy'].value_counts():
            df.loc['Dislike','Predicted Like'] += combine['Accuracy'].value_counts()['False Positive']
        if 'True Negative' in combine['Accuracy'].value_counts():
            df.loc['Dislike','Predicted Dislike'] += combine['Accuracy'].value_counts()['True Negative']

cm(confusion_matrix)

In [28]:
print (confusion_matrix)
print ("Accuracy: " + str((confusion_matrix.loc["Like","Predicted Like"]+confusion_matrix.loc["Dislike","Predicted Dislike"])/(confusion_matrix.loc["Like","Predicted Like"]+confusion_matrix.loc["Dislike","Predicted Dislike"]+confusion_matrix.loc["Like","Predicted Dislike"]+confusion_matrix.loc["Dislike","Predicted Like"])))
print ("True Positive " + str(confusion_matrix.loc["Like","Predicted Like"]/(confusion_matrix.loc["Like","Predicted Like"]+confusion_matrix.loc["Like","Predicted Dislike"])))
print ("True Negative " + str(confusion_matrix.loc["Dislike","Predicted Dislike"]/(confusion_matrix.loc["Dislike","Predicted Dislike"]+confusion_matrix.loc["Dislike","Predicted Like"])))
print ("False Positive " + str(confusion_matrix.loc["Dislike","Predicted Like"]/(confusion_matrix.loc["Dislike","Predicted Like"]+confusion_matrix.loc["Dislike","Predicted Dislike"])))
print ("False Negative " + str(confusion_matrix.loc["Like","Predicted Dislike"]/(confusion_matrix.loc["Like","Predicted Dislike"]+confusion_matrix.loc["Like","Predicted Like"])))
       

         Predicted Like  Predicted Dislike
Like               1240               2309
Dislike            1311               7701
Accuracy: 0.711806384842
True Positive 0.349394195548
True Negative 0.854527296937
False Positive 0.145472703063
False Negative 0.650605804452
